Fill in workbook_name variable under if __name__ == "__main__":

specify path inside links_file_path = r'path'

In [ ]:
import datetime
import pytz
import openpyxl


def get_data(output_file_path, workbook_name, start_date_user, end_date_user, start_timezone, tasks_timezone):
    try:
        schedule_workbook = openpyxl.load_workbook(output_file_path, data_only=True)
        schedule_sheet = schedule_workbook[workbook_name]

        schedule_workbook_index = schedule_workbook.sheetnames.index(workbook_name)

        adjacent_sheet_names = [schedule_workbook.sheetnames[i] 
                    for i in range(max(0, schedule_workbook_index - 1), 
                                                min(len(schedule_workbook.sheetnames), schedule_workbook_index + 2))]

        adjacent_sheets = [schedule_workbook[sheet_name] for sheet_name in adjacent_sheet_names]

        cr_number_col = get_column_index(schedule_sheet, "CR number")
        collection_col = get_column_index(schedule_sheet, "Collection")
        Start_Date_col = get_column_index(schedule_sheet, "Start Date")
        
        for schedule_sheet in adjacent_sheets:

            print(f'\n chosen sheet: {schedule_sheet.title} \n')

            for row_num in range(2, schedule_sheet.max_row + 1):
                
                cr_number = schedule_sheet.cell(row=row_num, column=cr_number_col).value
                collection = schedule_sheet.cell(row=row_num, column=collection_col).value
                start_date = schedule_sheet.cell(row=row_num, column=Start_Date_col).value


                # Skip rows containing delimter of table
                if cr_number is not None and any(substring in cr_number for substring in ['Count', 'CR number', 'Extra', 'for 1 per', 'CURRENTLY TESTING', 'CURRENTLY STOPPED', 'NO ACCESS TO DEVICES', 'CRs created/left:']):
                    continue

                if cr_number is None and start_date is None:
                    continue

                if start_date is not None and start_date.strip() == 'PatchTuesday Date:':
                    continue

                if start_date is None:

                    print(f"Warning: Start date {start_date} is empty in row {row_num}.")
                    continue

                try:
                    start_date_object = datetime.datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
                    start_date_localized = start_timezone.localize(start_date_object)
                    start_date_sfd = start_date_localized.astimezone(tasks_timezone)
                    formatted_start_date = start_date_sfd.strftime("%m/%d/%Y %H:%M")

                    # Convert string values to datetime.date objects
                    start_date_value = datetime.datetime.strptime(start_date.split()[0], "%Y-%m-%d").date()
                
                except ValueError:
                    print(f"Warning: Invalid date format in row {row_num}.")
                    continue

                if not (start_date_user <= start_date_value <= end_date_user):
                    continue  # Skip rows not within the specified date range

                if cr_number is None and start_date_user <= start_date_value <= end_date_user: 
                    print(f"Warning: CR number is None in row {row_num}.")
        
                fill_tasks(input_file_path, cr_number, collection, formatted_start_date)
                print(f"{cr_number} - {collection} - {formatted_start_date} - Updated")

    except Exception as e:
        print(f"Exception 1: An error occurred: {e}")

def get_column_index(sheet, column_name):
    for col_num in range(1, sheet.max_column + 1):
        if sheet.cell(row=1, column=col_num).value == column_name:
            return col_num
    raise ValueError(f"Column '{column_name}' not found in the sheet")

def fill_tasks(input_file_path, cr_number, collection, formatted_start_date):

    try:

        workbook = openpyxl.load_workbook(input_file_path, data_only=True)
        worksheet = workbook['All Tasks']

        # Get column indices
        group_name_col = None
        change_request_col = None
        planned_start_date_col = None

        for tasks_column_idx in range(1, worksheet.max_column + 1):
            column_value = worksheet.cell(row=1, column=tasks_column_idx).value
            if column_value:
                column_value_lower = column_value.lower()
                if column_value_lower == 'collection':
                    group_name_col = tasks_column_idx
                elif column_value_lower == 'cr number':
                    change_request_col = tasks_column_idx
                elif column_value_lower == 'start date':
                    planned_start_date_col = tasks_column_idx
        
        if change_request_col is None:
            print("Error: 'CR number' not found.")
            return
        elif group_name_col is None:
            print("Error: 'Collection' not found.")
            return
        elif planned_start_date_col is None:
            print("Error: 'Start Date' not found.")
            return
        
        collection_found = False

        # Update columns for the relevant rows
        for row_idx in range(2, worksheet.max_row + 1):
            group_name = worksheet.cell(row=row_idx, column=group_name_col).value
            if group_name and group_name.lower().strip().startswith(collection.lower()):
                collection_found = True
                worksheet.cell(row=row_idx, column=change_request_col).value = cr_number
                worksheet.cell(row=row_idx, column=planned_start_date_col).value = formatted_start_date

        if not collection_found:
            raise ValueError(f"Error: Collection '{collection}' not found.")

        # Save the changes
        workbook.save(input_file_path)
    
    except Exception as e:
        print(f"An error occurred: {e}")
     

if __name__ == "__main__":
    
    start_timezone = pytz.timezone('Europe/London')
    tasks_timezone = pytz.timezone('America/Los_Angeles')
    workbook_name = 'July2024'

    links_file_path = r'C:\Users\pelyu\OneDrive\Рабочий стол\chg\git\linkspath.txt'

    with open(links_file_path, 'r', encoding='utf-8') as file:
    # Read the lines
        lines = file.readlines()

    output_file_path = lines[0].strip() if len(lines) > 0 else ''
    input_file_path = lines[2].strip() if len(lines) > 2 else ''


    # Prompt the user to input start and end dates
    start_date_input = input("Enter the start date (YYYY-MM-DD): ")
    end_date_input = input("Enter the end date (YYYY-MM-DD): ")

    # Convert user input to datetime objects
    start_date_user = datetime.datetime.strptime(start_date_input, "%Y-%m-%d").date()
    end_date_user = datetime.datetime.strptime(end_date_input, "%Y-%m-%d").date()
    print(f'{start_date_user} - {end_date_user}')
   

    get_data(output_file_path, workbook_name, start_date_user, end_date_user, start_timezone, tasks_timezone)
